In [ ]:
##################################################################
# Loading adaptor with base model and inferencing 
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import PeftModel, PeftConfig

adapter_path = "C:\\Users\\Public\\Rag_based_chatbot\\Medical_bot\\Models\\lora_adapters"  # Replace with your actual path
configs = PeftConfig.from_pretrained(adapter_path)

# Load base model (4B version)
model_id = "google/medgemma-4b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cpu",
    torch_dtype=torch.float32,
    low_cpu_usage = True
)



c:\Users\Public\Rag_based_chatbot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [02:39<00:00, 79.82s/it]


In [ ]:
# Merge base model with adapters
model = PeftModel.from_pretrained(model, adapter_path, device_map={"": "cpu"}  # Explicit CPU mapping  
                                  )
model = model.merge_and_unload()  # Merge adapters into base model

def generate_response(prompt, max_new_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to('cpu')
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        # top_k=50,
        # top_p=0.95
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Medical prompt example
prompt = """<start_of_turn>user
I Had high fever. What are the potential causes?<end_of_turn>
<start_of_turn>assistant
"""

print(generate_response(prompt))
# Save merged model

# takes 45min

user
I Had high fever. What are the potential causes?
assistant
Hi,It is good that you are inquiring about the fever. It is a good sign that you are taking care of yourself. The fever can be due to various inflammatory conditions. Please get the blood reports done (CBC, ESR, CRP, LFT, LFT, RFT, RTG) and consult your doctor to know the possible causes. The underlying cause can be treated with medicines. Hope this helps.


In [4]:

print('Saving the model')
merged_model_path = "C:\\Users\\Public\\Rag_based_chatbot\\Medical_bot\\Models\\medgemma-4b-it-finetuned-merged-new"
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

# Save in safetensors format
model.save_pretrained(merged_model_path, safe_serialization=True)
###############################################################

Saving the model


### Running from merged-model

In [5]:
import os
os.chdir('c://Users//Public//Rag_based_chatbot')
os.getcwd()

'c:\\Users\\Public\\Rag_based_chatbot'

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# Option 1: Use raw string
# model = AutoModelForCausalLM.from_pretrained(r'Medical_bot\Models\medgemma-4b-it-finetuned-merged-new').cpu()
# tokenizer = AutoTokenizer.from_pretrained(r'Medical_bot\Models\medgemma-4b-it-finetuned-merged-new')

# Option 2: Use forward slashes
model = AutoModelForCausalLM.from_pretrained('Medical_bot\Models\medgemma-4b-it-finetuned-merged-new').cpu()
tokenizer = AutoTokenizer.from_pretrained('Medical_bot\Models\medgemma-4b-it-finetuned-merged-new')


<>:7: SyntaxWarning: invalid escape sequence '\M'
<>:8: SyntaxWarning: invalid escape sequence '\M'
<>:7: SyntaxWarning: invalid escape sequence '\M'
<>:8: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Admin\AppData\Local\Temp\ipykernel_22928\2087939396.py:7: SyntaxWarning: invalid escape sequence '\M'
  model = AutoModelForCausalLM.from_pretrained('Medical_bot\Models\medgemma-4b-it-finetuned-merged-new').cpu()
C:\Users\Admin\AppData\Local\Temp\ipykernel_22928\2087939396.py:8: SyntaxWarning: invalid escape sequence '\M'
  tokenizer = AutoTokenizer.from_pretrained('Medical_bot\Models\medgemma-4b-it-finetuned-merged-new')
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  7.06it/s]


In [ ]:
# Medical prompt example
prompt = """<start_of_turn>user
I Had high fever. What are the potential causes?<end_of_turn>
<start_of_turn>assistant
"""
inputs = tokenizer(prompt, return_tensors="pt").to('cpu')
outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        # top_k=50,
        # top_p=0.95
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# takes 80min 


user
I Had high fever. What are the potential causes?
assistant
Hi, As you had high fever, you may have viral or bacterial infection. Such as - Viral fever - Dengue fever - Typhoid fever - Influenza - Urinary tract infection - Ear infection - Sinus infection - Meningitis - Pneumonia - Malaria - Tuberculosis.  I would recommend you to take blood tests, urine tests, and x-ray reports to find out the exact cause of fever. You may also need to consult your doctor for proper treatment. Hope I have answered your query. Let me know if I can assist you further. Regards, Dr. Sumanth MBBS, DCH, DM
